In [55]:
import numpy as np
from skimage import io, color, transform

In [56]:
def resize_image(img, size, padding=255):
    '''
    Resize the given image to specified size
    '''
    # Append zeros or ones based on the choice of padding
    # to maintain the aspect ratio of the image
    rows, cols = img.shape[:2]
    max_size = max(rows, cols)
    canvas = np.ones((max_size, max_size), np.uint8) * padding
    
    # Place the actual image at the center of the canvas
    c_x, c_y = max_size / 2, max_size / 2
    x_beg = max(0, c_x - (cols / 2))
    y_beg = max(0, c_y - (rows / 2))
    x_end, y_end = x_beg + cols, y_beg + rows
    canvas[y_beg: y_end, x_beg: x_end] = img
        
    # Resize the image and write to op_dir_path specified
    reszd_img = transform.resize(canvas, (size, size))
    reszd_img = np.asarray(reszd_img * 255, np.uint8)
    return reszd_img

In [57]:
def read_data(csv_file_path):
    '''
    Read the training data and return X and y
    Returns:
        X = Training samples (m - samples of n features)
        y = Lables corresponding to each sample (size - m)
    '''
    data = np.loadtxt(csv_file_path, delimiter=',')
    rows, cols = data.shape
    y = data[:, -1].astype(np.int)
    X = data[:, :cols-1]
    return X, y

In [58]:
from sklearn import svm

In [59]:
def train_svm(train_data_file):
    '''
    Train an SVM model for the given X and y
    '''
    clf = svm.SVC()
    
    # Get the training data
    X, y = read_data(train_data_file)
    clf.fit(X, y)
    return clf

In [60]:
import pickle

In [61]:
def test_svm(model, test_data_file):
    '''
    Test the trained model with the test data
    '''
    X, y = read_data(test_data_file)
    pred_acc_list = []
    for sample, label in zip(X, y):
        pred = model.predict(sample.reshape(1, -1))[0]
        pred_acc_list.append(int(pred) == int(label))
    accuracy = sum(pred_acc_list) / float(len(pred_acc_list)) * 100
    print 'The Acccuracy of the model is = ', accuracy
    return accuracy

In [62]:
def nut_or_bolt(model, img_file_path):
    '''
    Given an image file, predicts and returns the kind
    Returns:
        Nut or Bolt (a string)
    '''
    img = io.imread(img_file_path)
    img = color.rgb2gray(img)
    img = np.asarray(img * 255, np.uint8)
    reszd_img = resize_image(img, 50).astype(np.float) / 255.0
    
    ip_data = reszd_img.flatten()
    pred = model.predict(ip_data.reshape(1, -1))
    print pred
    result = 'Nut' if not pred[0] else 'Bolt'
    return result

In [63]:
train_data_file = '/home/dhivakar/work/learning/events/anthill17/data/nuts_and_bolts_train.csv'
model_file_path = '/home/dhivakar/work/learning/events/anthill17/models/nut_n_bolt.pkl'
test_img_file = '/home/dhivakar/work/learning/events/anthill17/data/test/0015.jpg'

# Train the model and store it in a file. It can be loaded anytime later
model = train_svm(train_data_file)
pickle.dump(model, open(model_file_path, 'w'))

# Take an image and show the prediction + image
loaded_model = pickle.load(open(model_file_path, 'r'))
result = nut_or_bolt(loaded_model, test_img_file)
print result

# Calculate the accuracy of the model
accuracy = test_svm(loaded_model, train_data_file)

[0]
Nut
The Acccuracy of the model is =  92.0792079208
